# Description du problème

On se met à la place d'un développeur qui a produit une application mais ne l'a pas encore mis surl'App store Google sur lequel il connait:

-Nom 

-Genre

-Taille

-Prix

-Age du public visé

-Version d'Android

il cherche à connaitre le **nombre de télechargements** et **les critiques** que cette application aura lorsqu'il mettra son app sur l'App Store.

# Modèle

# Code

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [2]:
import sys
sys.path.append('../')  

data = pd.read_csv("GoogleApps/googleplaystore.csv",encoding='utf-8')
data.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


On modifie les données-> 1ere étape, on enlève les colonnes qui ne nous interesse pas: 

Les colonnes Rating, Reviews, Type (l'information est déjà indiquée dans la colonne Price), Genres (qui est assez similaire à la categorie), Last Updated et Current Ver (l'app n'est pas encore sortie donc c'est forcément la version 1.0).

In [3]:
colonnesAEliminer=["Reviews","Type","Genres","Last Updated","Current Ver"]
for colonne in colonnesAEliminer:
    data=data.drop(colonne, axis=1)
data.head()

,App,Category,Rating,Size,Installs,Price,Content Rating,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,19M,"10,000+",0,Everyone,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,14M,"500,000+",0,Everyone,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,8.7M,"5,000,000+",0,Everyone,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,25M,"50,000,000+",0,Teen,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,2.8M,"100,000+",0,Everyone,4.4 and up


La deuxième étape est de catégoriser les données pour faciliter leur manipulation : on remplace les valeurs par des nombres pour pouvoir utiliser les fonctions.

Pour Size, la catégorisation consistera à remplacer les valeurs de la colonnes par un float représentant la taille de l'app en Mo, Pour Install, on remplace les valeurs par un numero de classe (pour pouvoir appliquer nos fonctions du percpetron...) il y a 9 classes (1ere classe: de 0 à 100 install, 2eme classe : de 100 à 1000, 3eme classe: de 1000 à 10000 ...). Pour le prix, on remplace par le numero d'une classe, il y a 5 classes différentes en fonction de la hauteur du prix. Pour les noms des apps, on les remplace par leur longueur (c'est assez arbitraire mais un nom long peut plus facilement retenir l'attention ?). Pour les autres colonnes, on remplace chaque valeur par un identifiant, par ex, toute les app fonctionnant sur Android 4.4.2, aura comme valeur 7 dans la colonne.

On remplace aussi les "nan" dans les colonnes pour faciliter le remplacement des valeurs. 

In [4]:
def categorisation(val):
    global valeur
    if(val==valeur):
        return 1
    return 0

data=data.fillna("Empty")
def categorisationRating(val):
    if(val=="Empty" or int(val)==19):
        return 3
    return int(val)

#on ne renvoie que la principale version Androide par ex: si la version est 4.3 and up, on renvoie 4
def categorisationAndroid(val):
    if(val=="Varies with device" or val=="Empty"):
        return 4
    return int(val[0])

def categorisationName(val):
    return len(val)

listeCategory=np.unique(data["Category"].values).tolist()
def categorisationCategory(val):
    return listeCategory.index(val)+1
    
def categorisationPrice(val):
    if(val=="0" or val=="Everyone"):
        return 1
    else:
        val=float(val[1:])
        if(val<2):
            return 2
        elif(val<10):
            return 3
        elif(val<50):
            return 4
        else:
            return 5

def categorisationSize(val):
    if(type(val)==type("string")):
        if(val=="Varies with device"):
            return 25
        if(val[-1:]=='M'):
            coeff=1
        elif(val[-1:]=='K' or val[-1:]=='k'):
            coeff=0.001
        else:
            coeff=1
        val=val[:-1].replace(',','')
        return int(float(val)*coeff)
    return val

#renvoie la classe 1 à 9 (nb d'installation max est 1000000000)
def categorisationInstalls(val): 
    if(val=="Free"):
        val=0
    elif(type(val)==type("string")):
        val=val.replace(',','')
        val=val.replace('+','')
    v=int(val)
    if(v==0):
        v=1
    return max(1,int(math.log10(v)))

In [5]:
new_size=data["Size"].apply(categorisationSize)
data["Size"]=new_size.values

new_rating=data["Rating"].apply(categorisationRating)
data["Rating"]=new_rating.values

new_install=data["Installs"].apply(categorisationInstalls)
data["Installs"]=new_install.values

new_price=data["Price"].apply(categorisationPrice)
data["Price"]=new_price.values

new_name=data["App"].apply(categorisationName)
data["App"]=new_name.values

new_android=data["Android Ver"].apply(categorisationAndroid)
data["Android Ver"]=new_android.values


In [6]:
data.to_csv("GoogleApps/googleplaystoreCLEANpb2.csv")

In [7]:
for colonne in ["Category","Price","Android Ver","Content Rating"]:
    for val in np.unique(data[colonne].values):
        valeur=val
        new_rating=data[colonne].apply(categorisation)
        data[colonne+str(val)]=new_rating.values
    data=data.drop(colonne,axis=1)
data.tail(50)

,App,Rating,Size,Installs,Category1.9,CategoryART_AND_DESIGN,CategoryAUTO_AND_VEHICLES,CategoryBEAUTY,CategoryBOOKS_AND_REFERENCE,CategoryBUSINESS,...,Android Ver6,Android Ver7,Android Ver8,Content RatingAdults only 18+,Content RatingEmpty,Content RatingEveryone,Content RatingEveryone 10+,Content RatingMature 17+,Content RatingTeen,Content RatingUnrated
10791,17,4,38,4,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
10792,22,4,16,6,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
10793,9,4,78,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
10794,7,3,5,3,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
10795,24,4,4,5,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
10796,48,4,7,6,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
10797,21,4,46,6,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
10798,20,3,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
10799,14,4,6,5,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
10800,9,4,12,3,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [8]:
data.to_csv("GoogleApps/googleplaystoreCLEANpb1.csv")